# NLP Project: Extractive Summarization with BiLSTM + Attention

## Project Goal
- Build an extractive summarization model for messy study notes, lecture transcripts, and textbook excerpts.
- **Core Model**: BiLSTM + Attention.
- **Datasets**: Webis-TLDR-17 + WikiHow.

## Methodology
1. **Data Loading**: Load and combine datasets.
2. **Preprocessing**: Convert abstractive summaries to extractive labels (Oracle extraction).
3. **Tokenization**: Use BERT tokenizer.
4. **Model**: Word Embeddings -> BiLSTM -> Attention -> Sentence Importance Score.
5. **Training**: Binary Cross Entropy Loss.
6. **Evaluation**: ROUGE scores.

## 1. Install Dependencies
Note: For GPU support on Windows/Linux, we specify the CUDA version for PyTorch.

In [1]:
%pip install datasets transformers rouge-score nltk
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ----- ---------------------------------- 0.5/4.1 MB 3.4 MB/s eta 0:00:02
     --------------- ------------------------ 1.6/4.1 MB 3.8 MB/s eta 0:00:01
     -------------------- ------------------- 2.1/4.1 MB 3.8 MB/s eta 0:00:01
     --------------------------- ------------ 2.9/4.1 MB 3.6 MB/s eta 0:00:01
     ----------------------------------- ---- 3.7/4.1 MB 3.6 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 3.6 MB/s  0:00:01
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ----- ---------------------------------- 0.5/4.1 MB 3.4 MB/s eta 0:00:02
     ------------ --------------------------- 1.3/4.1 MB 3.7 MB/s eta 0:00:01
     -------------------- -----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Imports & Setup

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer
from rouge_score import rouge_scorer
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab') # Required for newer NLTK versions

c:\Users\moras\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\moras\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 3. Load Datasets
We load Webis-TLDR-17 and WikiHow. We ensure both have uniform columns: `text` and `summary`.
Note: If WikiHow is unavailable, we fall back to CNN/DailyMail for demonstration.

In [3]:
# Configuration
MAX_SAMPLES = 5000 # Limit samples for Colab demo speed. Increase for full training.

# 1. Load Webis-TLDR-17
print("Loading Webis-TLDR-17...")
try:
    dataset_webis = load_dataset("webis/tldr-17", split="train[:5%]") # Taking a slice
except Exception as e:
    print(f"Webis dataset load failed: {e}. Proceeding without it.")
    dataset_webis = None

# 2. Load WikiHow (with Fallback)
print("Loading WikiHow...")
try:
    # WikiHow is sometimes flaky on HF. We try to load it.
    dataset_wikihow = load_dataset("wikihow", "all", split="train[:10%]")
except Exception as e:
    print(f"WikiHow load failed: {e}. Switching to CNN/DailyMail as fallback for valid execution.")
    dataset_wikihow = load_dataset("cnn_dailymail", "3.0.0", split="train[:10%]")

# 3. Unify Columns
def unify_columns(dataset, text_col, summary_col):
    # Keep only relevant columns to avoid conflicts
    cols_to_keep = [text_col, summary_col]
    dataset = dataset.remove_columns([c for c in dataset.column_names if c not in cols_to_keep])
    dataset = dataset.rename_column(text_col, "text")
    dataset = dataset.rename_column(summary_col, "summary")
    return dataset

if dataset_webis:
    if 'content' in dataset_webis.column_names:
        dataset_webis = unify_columns(dataset_webis, 'content', 'summary')
    elif 'body' in dataset_webis.column_names:
        dataset_webis = unify_columns(dataset_webis, 'body', 'summary')

# Check for WikiHow or Fallback columns
if 'headline' in dataset_wikihow.column_names: # WikiHow
    dataset_wikihow = unify_columns(dataset_wikihow, 'text', 'headline')
elif 'highlights' in dataset_wikihow.column_names: # CNN/DM fallback
    dataset_wikihow = unify_columns(dataset_wikihow, 'article', 'highlights')

# 4. Concatenate
if dataset_webis:
    combined_dataset = concatenate_datasets([dataset_webis, dataset_wikihow])
else:
    combined_dataset = dataset_wikihow

# Shuffle and limit
combined_dataset = combined_dataset.shuffle(seed=42).select(range(min(len(combined_dataset), MAX_SAMPLES)))

print(f"Combined Dataset Size: {len(combined_dataset)}")
print("Sample:", combined_dataset[0])

Loading Webis-TLDR-17...
Webis dataset load failed: Dataset scripts are no longer supported, but found tldr-17.py. Proceeding without it.
Loading WikiHow...
WikiHow load failed: Dataset 'wikihow' doesn't exist on the Hub or cannot be accessed.. Switching to CNN/DailyMail as fallback for valid execution.
Combined Dataset Size: 5000
Sample: {'text': '(CNN) -- A small amount of radioactive gas escaped from a steam generator at Southern California\'s San Onofre nuclear power plant during a water leak, but there was no threat to public health, federal regulators said Wednesday. Operators shut down the plant\'s No. 3 reactor on Tuesday after the water leak was discovered, Nuclear Regulatory Commission spokesman Victor Dricks said. The gas was vented from the steam generator into an auxiliary building, where it triggered a radiation monitor, he said. The amount of gas that leaked was not immediately known, but it was a small amount that won\'t endanger the public or plant workers, Dricks said

## 4. Preprocessing: Oracle Label Generation
Since we are building an *extractive* summarizer but have *abstractive* ground truth, we need to generate labels (0 or 1) for each sentence in the source text.

We use a **Greedy Method**:
1. Split text into sentences.
2. Select sentences that maximize ROUGE score with the abstractive summary.

In [4]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def greedy_extraction(text, summary):
    # Split sentences
    sentences = nltk.sent_tokenize(text)
    summary_sentences = nltk.sent_tokenize(summary)
    
    # Clean/lowercase
    sentences = [s.strip() for s in sentences if len(s.split()) > 3] # Filter very short ones
    
    if not sentences:
        return [], []

    selected_indices = []
    current_summary = ""
    original_score = 0.0
    
    # Simple Greedy: Add sentence if it improves ROUGE-1/L F1
    # To speed up for Colab, we can do a simpler heuristic: Max individual overlap.
    
    # Fast Heuristic: Label 1 if sentence shares significant unigrams with summary
    labels = []
    for sent in sentences:
        # Calculate Rouge-1 Recall of this sentence vs summary
        scores = scorer.score(summary, sent)
        r1_rec = scores['rouge1'].recall
        rL_rec = scores['rougeL'].recall
        
        # Thresholding (Heuristic)
        # If sentence covers > 10% of summary or has high overlap
        if r1_rec > 0.1 or rL_rec > 0.1:
            labels.append(1)
        else:
            labels.append(0)
            
    return sentences, labels

# Test Preprocessing
test_txt = combined_dataset[0]['text']
test_sum = combined_dataset[0]['summary']
s, l = greedy_extraction(test_txt, test_sum)
print(f"Extracted {sum(l)}/{len(l)} sentences.")

Extracted 9/11 sentences.


## 5. Dataset Class & Tokenizer
We treat each **sentence** as a training sample for the BiLSTM.
This creates a dataset of labelled sentences.

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class ExtractiveDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer, max_len=50, limit=None):
        self.samples = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        print("Preprocessing data (this may take a while)...")
        count = 0
        for item in tqdm(hf_dataset):
            if limit and count >= limit: break
            
            text = item['text']
            summary = item['summary']
            
            sents, labels = greedy_extraction(text, summary)
            
            # Add sentences as individual samples
            for sent, label in zip(sents, labels):
                self.samples.append((sent, label))
            
            count += 1
            
        print(f"Processed {len(self.samples)} sentences.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sentence, label = self.samples[idx]
        
        # Tokenize
        encoding = self.tokenizer(
            sentence,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

# Create Dataset
dataset = ExtractiveDataset(combined_dataset, tokenizer, max_len=64)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

Preprocessing data (this may take a while)...


100%|██████████| 5000/5000 [02:21<00:00, 35.30it/s]

Processed 149228 sentences.


## 6. Model: BiLSTM + Attention
Standard Architecture:
1. **Embedding**: Convert token IDs to vectors.
2. **BiLSTM**: Encode sequence context.
3. **Attention**: Aggregate LSTM hidden states into a single sentence vector.
4. **Classifier**: Linear layer -> Sigmoid -> Importance Score.

In [6]:
class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(BiLSTMAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        
        # Attention Weights
        self.W_w = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.u_w = nn.Linear(hidden_dim * 2, 1, bias=False)
        
        # Classification Head
        self.fc = nn.Linear(hidden_dim * 2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        # [Batch, SeqLen, Embed]
        embedded = self.embedding(input_ids)
        
        # Pack padded sequence if desired, but for simplicity we run full LSTM with masking logic later if needed
        # LSTM: [Batch, SeqLen, Hidden*2]
        lstm_out, _ = self.lstm(embedded)
        
        # Attention Mechanism (Yang et al. 2016 style)
        # u = tanh(W * h)
        u = torch.tanh(self.W_w(lstm_out))
        # score = u * context_vector (u_w)
        scores = self.u_w(u) # [Batch, SeqLen, 1]
        
        # Masking padding (Very Important)
        if attention_mask is not None:
            mask = attention_mask.unsqueeze(-1) # [Batch, SeqLen, 1]
            scores = scores.masked_fill(mask == 0, -1e9)
            
        alpha = torch.softmax(scores, dim=1) # [Batch, SeqLen, 1]
        
        # Sentence Vector = sum(alpha * h)
        sentence_vector = torch.sum(lstm_out * alpha, dim=1) # [Batch, Hidden*2]
        
        # Prediction
        prediction = self.sigmoid(self.fc(sentence_vector))
        
        return prediction.squeeze()

## 7. Training
We train the model using BCELoss (since it's binary classification: Important vs Not Important).

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Explicit check for CUDA success
if device.type != 'cuda':
    print("WARNING: CUDA not found. Training will be slow. Ensure proper torch version installed.")
else:
    print(f"GPU: {torch.cuda.get_device_name(0)}")

model = BiLSTMAttention(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,    # Keep small for Colab RAM
    hidden_dim=256
).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 3

model.train()
for epoch in range(EPOCHS):
    epoch_loss = 0
    progress = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    
    for batch in progress:
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, mask)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        progress.set_postfix(loss=loss.item())
        
    print(f"Epoch {epoch+1} Average Loss: {epoch_loss/len(train_loader):.4f}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3060 Laptop GPU


Epoch 1: 100%|██████████| 4664/4664 [03:43<00:00, 20.85it/s, loss=0.367]


Epoch 1 Average Loss: 0.4714


Epoch 2: 100%|██████████| 4664/4664 [03:27<00:00, 22.43it/s, loss=0.599]


Epoch 2 Average Loss: 0.4377


Epoch 3: 100%|██████████| 4664/4664 [02:20<00:00, 33.09it/s, loss=0.317]

Epoch 3 Average Loss: 0.3969


## 8. Evaluation & Sample Predictions
We extract sentences with high scores and compare them (qualitatively) to the ground truth.

In [8]:
def summarize(text, model, tokenizer, device, threshold=0.5):
    model.eval()
    sentences = nltk.sent_tokenize(text)
    cleaned_sentences = [s.strip() for s in sentences if len(s.split()) > 3]
    
    if not cleaned_sentences: return "No content"
    
    inputs = tokenizer(
        cleaned_sentences,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=64
    )
    
    with torch.no_grad():
        input_ids = inputs['input_ids'].to(device)
        mask = inputs['attention_mask'].to(device)
        scores = model(input_ids, mask)
        
    # Select top ranked or threshold
    # Since 'scores' can be scalar if 1 sentence, handle that:
    if scores.ndim == 0:
        scores = scores.unsqueeze(0)
        
    scores = scores.cpu().numpy()
    
    # Sort indices by score
    top_indices = scores.argsort()[::-1]
    
    # Pick top 3 or threshold
    selected_sents = []
    for idx in top_indices:
        if scores[idx] > threshold or len(selected_sents) < 3:
            selected_sents.append(cleaned_sentences[idx])
            if len(selected_sents) >= 3: break
            
    return " ".join(selected_sents)

# Evaluate on a sample
test_idx = 0
sample_text = combined_dataset[test_idx]['text']
ground_truth = combined_dataset[test_idx]['summary']

pred_summary = summarize(sample_text, model, tokenizer, device)

print("Original Text Segment:", sample_text[:400], "...")
print("\nGold Summary:", ground_truth)
print("\nPredicted Summary:", pred_summary)

Original Text Segment: (CNN) -- A small amount of radioactive gas escaped from a steam generator at Southern California's San Onofre nuclear power plant during a water leak, but there was no threat to public health, federal regulators said Wednesday. Operators shut down the plant's No. 3 reactor on Tuesday after the water leak was discovered, Nuclear Regulatory Commission spokesman Victor Dricks said. The gas was vented ...

Gold Summary: A small amount of radioactive gas escaped from a steam generator, the NRC says .
The leak does not pose any threat to human health, an NRC spokesman says .
Operators shut down the No. 3 reactor at California's San Onofre plant as a result .

Predicted Summary: (CNN) -- A small amount of radioactive gas escaped from a steam generator at Southern California's San Onofre nuclear power plant during a water leak, but there was no threat to public health, federal regulators said Wednesday. The water leak occurred in the thousands of tubes that carry heated 

## 9. Evaluation Metrics (ROUGE)
Simple Loop to calculate average ROUGE scores on a small test set.

In [9]:
eval_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

r1, r2, rl = 0, 0, 0
N = 10

print(f"Evaluating on {N} samples...")
for i in range(N):
    src = combined_dataset[i]['text']
    ref = combined_dataset[i]['summary']
    
    hyp = summarize(src, model, tokenizer, device)
    
    scores = eval_scorer.score(ref, hyp)
    r1 += scores['rouge1'].fmeasure
    r2 += scores['rouge2'].fmeasure
    rl += scores['rougeL'].fmeasure

print(f"ROUGE-1: {r1/N:.4f}")
print(f"ROUGE-2: {r2/N:.4f}")
print(f"ROUGE-L: {rl/N:.4f}")

Evaluating on 10 samples...
ROUGE-1: 0.2866
ROUGE-2: 0.0843
ROUGE-L: 0.1654


In [11]:
MODEL_PATH = "bilstm_attention.pth"

# Save
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model weights saved to {MODEL_PATH}")


Model weights saved to bilstm_attention.pth
